In [ ]:
using OptimalTransport
using DelimitedFiles
using Distances
using ProgressMeter
using Megafauna
include("../src/DPC.jl")

dihedrals = readdlm("../data/ADP/A_2D_1ps.dat");
shifted_dehrals = readdlm("../data/ADP/A_2D_1ps_shifted.dat");
change_points = readdlm("../data/ADP/A_2D_1ps_shifted.lam10.0alpha0.7.transitionProba.dat");

change_points = findall(x -> any(y -> y > 0, x), eachrow(change_points[:,2:3]))

N = length(change_points)
N = 100
D = zeros(N,N)
ε = 1.
p = Progress((N * (N - 1)) ÷ 2)   # minimum update interval: 1 second

Threads.@threads for idx = 1:N^2
    i, j = fldmod1(idx, N)  # Convert linear index to (i,j) coordinates
    if i > j
        t0, t1 = change_points[i], change_points[i+1]
        s0, s1 = change_points[j], change_points[j+1]
        Si, Sj = dihedrals[t0:t1,:], dihedrals[s0:s1,:]
        C = pairwise(PeriodicEuclidean([360 360]), Si', Sj').^2
        a, b = fill(1, t1 - t0 + 1) / (t1 - t0 + 1), fill(1, s1 - s0 + 1) / (s1 - s0 + 1)
        d = sinkhorn2(a, b, C, ε);
        D[i,j] = D[j, i] = d
        next!(p)
    end
end

# change points were computed via SIMPLE, now compute intersegment distances with EMD 